# 1) Objetivo do projeto
O objetivo desse projeto de engenharia de dados é responder perguntas relacionadas a vendas de roupas no formato varejo. Deseja-se responder as questões abaixo:

- Qual foi a categoria de produto mais vendido em todo o período disponível para análise?
- Qual o mês com mais vendas?
- Qual cliente mais devolveu no mês de abril do ano mais recente?
- Os produtos com mais vendas no mês são os produtos com maior taxa de desconto?
- O mês com mais venda é o mês com maior incidência de desconto?

# 2) Coleta
## 2.1) Busca pelos dados
Iniciou-se o processo de busca pelo conjunto de dados no site www.kaggle.com.br, uma comunidade voltada para o aprendizado em ciência de dados. O conjunto de dados escolhido foi no tema de venda de roupas no setor varejo, disponível no link a seguir: https://www.kaggle.com/datasets/ricgomes/global-fashion-retail-stores-dataset

## 2.2) Coleta dos dados
Segundo a plataforma Kaggle, o conjunto de dados escolhido está na versão bronze e tem 6 arquivos em .csv. Foi escolhido um conjunto com mais de um arquivo para intensificar a prática da modelagem com o modelo estrela. Os dados simulam 2 anos de vendas de uma empresa multinacional de venda de roupas, totalizando 1GB de tamanho.

![imagem](https://i.ibb.co/PZtwfxj9/429613623-98618ccf-0961-4b5b-9627-0098b327289d.png)

Os arquivos foram baixados para o computador local para ser feito o upload no Community Databricks, plataforma sugerida e escolhida para produzir o projeto MVP.
O processo de upload no Community Databricks é iniciado com os comandos + New -> Add or upload data no canto superior esquerdo da plataforma.
![imagem](https://i.ibb.co/ffF01rF/429633441-26f32398-8668-4583-b02a-8d62d6d92e19.png)

Os 6 arquivos são escolhidos para serem carregados e salvos na pasta "global_fashion" para melhor organização no diretório DBFS. São eles: clientes.csv, descontos.csv, empregados.csv, produtos.csv, lojas.csv e transacoes.csv

![imagem](https://ibb.co/N2L4KSFb)

Com isso, todos os dados necessários já estão no Databricks para iniciar o projeto no notebook.

# 3) Modelagem

## 3.1) Modelo estrela

![Lógico_MVP1v1](https://i.ibb.co/TxVyX2Hw/432066156-e3be351c-2e04-4c59-ac6d-9efbef23805c.png)


## 3.2) Catálogo de dados
- Nome do conjunto de dados: GlobalFashionRetail
- Descrição do conjunto de dados: esse conjunto de dados simula 2 anos de vendas de roupas, no modo varejo, de uma loja presente em vários países
- Fonte: Kaggle
- Uso: é usada para analisar quantidade de vendas, desempenho das vendas em relação a promoções, vendas segmentadas por cliente e por local, entre outras aplicações


## 3.3) Dicionário de dados
Abaixo está o dicionário de dados por tabela existente no modelo

![transacoes](https://i.ibb.co/jvyDVGd2/432519940-acca1df6-e1c3-4b7f-9d1b-914283c63a0a.png)

![clientes](https://i.ibb.co/3mSY1T5Z/432520090-6f3f151c-398a-4b55-894a-484fc3850322.png)

![produtos](https://i.ibb.co/7d5JWynm/432520182-3e61a1b9-a323-4991-a669-fb58c476d275.png)

![funcionarios](https://i.ibb.co/PGmdVq83/432520258-56862200-47be-4ee1-9515-e08885ffa3eb.png)

![lojas](https://i.ibb.co/Mxt39qNF/432520295-ddddb4d0-449b-45a4-ac5d-52068a535f54.png)

# 4) Carga
## 4.1) Extração, transformação e carga

In [0]:
import pandas as pd
import io
import requests
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

In [0]:
# Iniciar a SparkSession
spark = SparkSession.builder.appName("ProjetoPySparkFashion").getOrCreate()

In [0]:
# Carregar os arquivos brutos que foram salvos no DBFS
transacoes = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/tables/global_fashion/transactions.csv")
clientes = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/tables/global_fashion/customers.csv")
descontos = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/tables/global_fashion/discounts.csv")
funcionarios = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/tables/global_fashion/employees.csv")
produtos = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/tables/global_fashion/products.csv")
lojas = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/tables/global_fashion/stores.csv")

In [0]:
%sql DROP DATABASE IF EXISTS fbronze CASCADE;

In [0]:
%sql CREATE DATABASE fbronze;

**Observação**:
o ideal seria manter os dados na camada bronze o mais fiel possível aos dados brutos, porém, como o database não aceita espaços em branco no nome dos campos, altera-se os nomes dos campos para criar as tabelas

In [0]:
from pyspark.sql.functions import col

# Removendo espaços nos nomes das colunas
transacoes = transacoes.select(
    [col(c).alias(c.replace(" ", "")) for c in transacoes.columns]
)

clientes = clientes.select(
    [col(c).alias(c.replace(" ", "")) for c in clientes.columns]
)
produtos = produtos.select(
    [col(c).alias(c.replace(" ", "")) for c in produtos.columns]
)
funcionarios = funcionarios.select(
    [col(c).alias(c.replace(" ", "")) for c in funcionarios.columns]
)
descontos = descontos.select(
    [col(c).alias(c.replace(" ", "")) for c in descontos.columns]
)
lojas = lojas.select(
    [col(c).alias(c.replace(" ", "")) for c in lojas.columns]
)


In [0]:
# Salvando as tabelas no banco de dados fbronze
transacoes.write.mode("overwrite").saveAsTable("fbronze.btransacoes")
clientes.write.mode("overwrite").saveAsTable("fbronze.bclientes")
produtos.write.mode("overwrite").saveAsTable("fbronze.bprodutos")
funcionarios.write.mode("overwrite").saveAsTable("fbronze.bfuncionarios")
descontos.write.mode("overwrite").saveAsTable("fbronze.bdescontos")
lojas.write.mode("overwrite").saveAsTable("fbronze.blojas")

In [0]:
# Usando SQL para visualizar as tabelas
vtransacoes = spark.sql("SELECT * FROM fbronze.btransacoes LIMIT 3")
vclientes = spark.sql("SELECT * FROM fbronze.bclientes LIMIT 3")
vprodutos = spark.sql("SELECT * FROM fbronze.bprodutos LIMIT 3")
vfuncionarios = spark.sql("SELECT * FROM fbronze.bfuncionarios LIMIT 3")
vdescontos = spark.sql("SELECT * FROM fbronze.bdescontos LIMIT 3")
vlojas = spark.sql("SELECT * FROM fbronze.blojas LIMIT 3")

# Exibir as tabelas
print("Transacões:")
display(vtransacoes)
print("Clientes:")
display(vclientes)
print("Produtos:")
display(vprodutos)
print("Funcionários:")
display(vfuncionarios)
print("Descontos:")
display(vdescontos)
print("Lojas:")
display(vlojas)

Transacões:


InvoiceID,Line,CustomerID,ProductID,Size,Color,UnitPrice,Quantity,Date,Discount,LineTotal,StoreID,EmployeeID,Currency,CurrencySymbol,SKU,TransactionType,PaymentMethod,InvoiceTotal
INV-US-001-03558761,1,47162,485,M,null,80.5,1,2023-01-01 15:42:00,0.0,80.5,1,7,USD,$,MASU485-M-,Sale,Cash,126.7
INV-US-001-03558761,2,47162,2779,G,null,31.5,1,2023-01-01 15:42:00,0.4,18.9,1,7,USD,$,CHCO2779-G-,Sale,Cash,126.7
INV-US-001-03558761,3,47162,64,M,NEUTRAL,45.5,1,2023-01-01 15:42:00,0.4,27.3,1,7,USD,$,MACO64-M-NEUTRAL,Sale,Cash,126.7


Clientes:


CustomerID,Name,Email,Telephone,City,Country,Gender,DateOfBirth,JobTitle
1,Tyler Garcia,tyler.garcia@fake_gmail.com,922.970.2265x47563,New York,United States,M,2003-07-15,null
2,Joshua Miller,joshua.miller@fake_gmail.com,+1-958-729-6169,New York,United States,M,2000-06-16,Records manager
3,Alison Marshall DDS,alison.marshall.dds@fake_hotmail.com,+1-645-567-0876x5409,New York,United States,F,2003-07-22,null


Produtos:


...

EmployeeID,StoreID,Name,Position
1,1,Stephen Johnson,Store Manager
2,1,Rebecca Myers,Assistant Manager
3,1,Katherine Buchanan,Cashier


Descontos:


Start,End,Discont,Description,Category,SubCategory
2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Feminine,Coats and Blazers
2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Feminine,Sweaters and Knitwear
2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Masculine,Coats and Blazers


Lojas:


StoreID,Country,City,StoreName,NumberofEmployees,ZIPCode,Latitude,Longitude
1,United States,New York,Store New York,10,10001,40.7128,-74.006
2,United States,Los Angeles,Store Los Angeles,8,90001,34.0522,-118.2437
3,United States,Chicago,Store Chicago,9,60601,41.8781,-87.6298


In [0]:
%sql DROP DATABASE IF EXISTS fsilver CASCADE;

In [0]:
%sql CREATE DATABASE fsilver;

In [0]:
from pyspark.sql.functions import to_timestamp

# Definindo os tipos dos campos de ftransacoes (fashiontransacoes)
ftransacoes = spark.table("fbronze.btransacoes")\
    .select(
        col("InvoiceID").cast("char(30)"),
        col("Line").cast("integer"),
        col("CustomerID").cast("integer"),
        col("ProductID").cast("integer"),
        col("Size").cast("char(10)"),
        col("Color").cast("char(30)"),
        col("UnitPrice").cast("float"),
        col("Quantity").cast("integer"),
        to_timestamp(col("Date")).alias("Date"),
        col("Discount").cast("float"),
        col("LineTotal").cast("float"),
        col("StoreID").cast("integer"),
        col("EmployeeID").cast("integer"),
        col("Currency").cast("char(10)"),
        col("CurrencySymbol").cast("char(5)"),
        col("SKU").cast("char(40)"),
        col("TransactionType").cast("char(20)"),
        col("PaymentMethod").cast("char(30)"),
        col("InvoiceTotal").cast("float")
    )

# Definindo os tipos dos campos de fclientes
fclientes = spark.table("fbronze.bclientes")\
    .select(
        col("CustomerID").cast("integer"),
        col("Name").cast("char(60)"),
        col("Email").cast("char(60)"),
        col("Telephone").cast("char(60)"),
        col("City").cast("char(30)"),
        col("Country").cast("char(30)"),
        col("Gender").cast("char(1)"),
        to_timestamp(col("DateOfBirth")).alias("DateOfBirth"),
        col("JobTitle").cast("char(60)")
    )

# Definindo os tipos dos campos de fprodutos
fprodutos = spark.table("fbronze.bprodutos")\
    .select(
        col("ProductID").cast("integer"),
        col("Category").cast("char(50)"),
        col("SubCategory").cast("char(50)"),
        col("DescriptionPT").cast("char(100)"),
        col("Color").cast("char(20)"),
        col("Sizes").cast("char(10)"),
        col("ProductionCost").cast("float")
    )

# Definindo os tipos dos campos de ffuncionarios
ffuncionarios = spark.table("fbronze.bfuncionarios")\
    .select(
        col("EmployeeID").cast("integer"),
        col("StoreID").cast("integer"),
        col("Name").cast("char(60)"),
        col("Position").cast("char(60)"),
    )

# Definindo os tipos dos campos de flojas
flojas = spark.table("fbronze.blojas")\
    .select(
        col("StoreID").cast("integer"),
        col("Country").cast("char(30)"),
        col("City").cast("char(30)"),
        col("StoreName").cast("char(30)"),
        col("NumberOfEmployees").cast("integer"),
        col("ZIPCode").cast("char(10)"),
        col("Latitude").cast("char(30)"),
        col("Longitude").cast("char(30)")
    )


In [0]:
# Salvando as tabelas com seus campos com os tipos definidos na camada silver
ftransacoes.write.mode("overwrite").saveAsTable("fsilver.stransacoes")
fclientes.write.mode("overwrite").saveAsTable("fsilver.sclientes")
fprodutos.write.mode("overwrite").saveAsTable("fsilver.sprodutos")
ffuncionarios.write.mode("overwrite").saveAsTable("fsilver.sfuncionarios")
flojas.write.mode("overwrite").saveAsTable("fsilver.slojas")

In [0]:
# Usando SQL para visualizar as tabelas criadas
vtransacoes = spark.sql("SELECT * FROM fsilver.stransacoes LIMIT 3")
vclientes = spark.sql("SELECT * FROM fsilver.sclientes LIMIT 3")
vprodutos = spark.sql("SELECT * FROM fsilver.sprodutos LIMIT 3")
vfuncionarios = spark.sql("SELECT * FROM fsilver.sfuncionarios LIMIT 3")
vlojas = spark.sql("SELECT * FROM fsilver.slojas LIMIT 3")

# Exibir as tabelas
print("Transacões:")
display(vtransacoes)
print("Clientes:")
display(vclientes)
print("Produtos:")
display(vprodutos)
print("Funcionários:")
display(vfuncionarios)
print("Lojas:")
display(vlojas)

Transacões:


InvoiceID,Line,CustomerID,ProductID,Size,Color,UnitPrice,Quantity,Date,Discount,LineTotal,StoreID,EmployeeID,Currency,CurrencySymbol,SKU,TransactionType,PaymentMethod,InvoiceTotal
INV-US-001-03558761,1,47162,485,M,null,80.5,1,2023-01-01T15:42:00.000+0000,0.0,80.5,1,7,USD,$,MASU485-M-,Sale,Cash,126.7
INV-US-001-03558761,2,47162,2779,G,null,31.5,1,2023-01-01T15:42:00.000+0000,0.4,18.9,1,7,USD,$,CHCO2779-G-,Sale,Cash,126.7
INV-US-001-03558761,3,47162,64,M,NEUTRAL,45.5,1,2023-01-01T15:42:00.000+0000,0.4,27.3,1,7,USD,$,MACO64-M-NEUTRAL,Sale,Cash,126.7


Clientes:


CustomerID,Name,Email,Telephone,City,Country,Gender,DateOfBirth,JobTitle
1,Tyler Garcia,tyler.garcia@fake_gmail.com,922.970.2265x47563,New York,United States,M,2003-07-15T00:00:00.000+0000,null
2,Joshua Miller,joshua.miller@fake_gmail.com,+1-958-729-6169,New York,United States,M,2000-06-16T00:00:00.000+0000,Records manager
3,Alison Marshall DDS,alison.marshall.dds@fake_hotmail.com,+1-645-567-0876x5409,New York,United States,F,2003-07-22T00:00:00.000+0000,null


Produtos:


ProductID,Category,SubCategory,DescriptionPT,Color,Sizes,ProductionCost
1,Feminine,Coats and Blazers,Esportivo Veludo Verde Com Botões,null,S|M|L|XL,10.73
2,Feminine,Sweaters and Knitwear,Luxuoso Denim Rosa Com Botões,PINK,S|M|L|XL,19.55
3,Feminine,Dresses and Jumpsuits,Retrô Tricot Preto Estampado,BLACK,S|M|L|XL,25.59


Funcionários:


EmployeeID,StoreID,Name,Position
1,1,Stephen Johnson,Store Manager
2,1,Rebecca Myers,Assistant Manager
3,1,Katherine Buchanan,Cashier


Lojas:


StoreID,Country,City,StoreName,NumberOfEmployees,ZIPCode,Latitude,Longitude
1,United States,New York,Store New York,10,10001,40.7128,-74.006
2,United States,Los Angeles,Store Los Angeles,8,90001,34.0522,-118.2437
3,United States,Chicago,Store Chicago,9,60601,41.8781,-87.6298


# 5) Análise

## 5.1) Qualidade dos dados

### 5.1.1) Chaves nulas
Checar valores nulos nas chaves das tabelas, começando por fsilver.stransacoes, e verificando todas as chaves das demais tabelas

In [0]:
%sql
--Checar valores nulos nas chaves das tabelas, primeiro em stransacoes
SELECT 'CustomerID' AS campo, COUNT(*) AS total_nulos 
FROM fsilver.stransacoes 
WHERE CustomerID IS NULL OR CustomerID < 0

campo,total_nulos
CustomerID,0


In [0]:
# Verificar as chaves das tabelas fsilver.stransacoes para garantir que não há chaves nulas

# Executar as consultas com Spark e armazenar os resultados, primeiro para a tabela fsilver.stransacoes e depois para as demais
df_customer_nulls = spark.sql("SELECT 'CustomerID' AS campo, COUNT(*) AS total_nulos FROM fsilver.stransacoes WHERE CustomerID IS NULL")
df_invoice_nulls = spark.sql("SELECT 'InvoiceID' AS campo, COUNT(*) AS total_nulos FROM fsilver.stransacoes WHERE InvoiceID IS NULL")
df_product_nulls = spark.sql("SELECT 'ProductID' AS campo, COUNT(*) AS total_nulos FROM fsilver.stransacoes WHERE ProductID IS NULL")
df_store_nulls = spark.sql("SELECT 'StoreID' AS campo, COUNT(*) AS total_nulos FROM fsilver.stransacoes WHERE StoreID IS NULL")
df_employee_nulls = spark.sql("SELECT 'EmployeeID' AS campo, COUNT(*) AS total_nulos FROM fsilver.stransacoes WHERE EmployeeID IS NULL")

# Exibir os resultados
display(df_customer_nulls)
display(df_invoice_nulls)
display(df_product_nulls)
display(df_store_nulls)
display(df_employee_nulls)

campo,total_nulos
CustomerID,0


campo,total_nulos
InvoiceID,0


campo,total_nulos
ProductID,0


campo,total_nulos
StoreID,0


campo,total_nulos
EmployeeID,0


In [0]:
df_customer_nulls2 = spark.sql("SELECT 'CustomerID' AS campo, COUNT(*) AS total_nulos FROM fsilver.sclientes WHERE CustomerID IS NULL")

display(df_customer_nulls2)

campo,total_nulos
CustomerID,0


In [0]:
df_product_nulls2 = spark.sql("SELECT 'ProductID' AS campo, COUNT(*) AS total_nulos FROM fsilver.sprodutos WHERE ProductID IS NULL")

display(df_product_nulls2)

campo,total_nulos
ProductID,0


In [0]:
df_employee_nulls2 = spark.sql("SELECT 'EmployeeID' AS campo, COUNT(*) AS total_nulos FROM fsilver.sfuncionarios WHERE EmployeeID IS NULL")
df_store_nulls2 = spark.sql("SELECT 'StoreID' AS campo, COUNT(*) AS total_nulos FROM fsilver.sfuncionarios WHERE StoreID IS NULL")

display(df_employee_nulls2)
display(df_store_nulls2)

campo,total_nulos
EmployeeID,0


campo,total_nulos
StoreID,0


In [0]:
df_store_nulls3 = spark.sql("SELECT 'StoreID' AS campo, COUNT(*) AS total_nulos FROM fsilver.slojas WHERE StoreID IS NULL")

display(df_store_nulls3)

campo,total_nulos
StoreID,0


**Obs1**.: verificado que não há chaves nulas

### 5.1.2) Anos disponíveis
Checar se há mais dados do que 2 anos inteiros e ano ou mês maior do que o ano atual ou muito antigo para eliminar, pois não será necessário na análise

In [0]:
%sql
SELECT YEAR(Date) AS anos_disponiveis, MONTH(Date) AS meses_disponiveis
FROM fsilver.stransacoes
GROUP BY anos_disponiveis, meses_disponiveis
ORDER BY anos_disponiveis, meses_disponiveis

anos_disponiveis,meses_disponiveis
2023,1
2023,2
2023,3
2023,4
2023,5
2023,6
2023,7
2023,8
2023,9
2023,10


In [0]:
%sql
--Eliminar as transações do ano 2025 da tabela
DELETE FROM fsilver.stransacoes 
WHERE YEAR(Date) = 2025

num_affected_rows
456353


In [0]:
%sql
-- Verificando que as transacoes do ano 2025 foram excluídas
SELECT YEAR(Date) AS anos_disponiveis, MONTH(Date) AS meses_disponiveis
FROM fsilver.stransacoes
GROUP BY anos_disponiveis, meses_disponiveis
ORDER BY anos_disponiveis, meses_disponiveis

anos_disponiveis,meses_disponiveis
2023,1
2023,2
2023,3
2023,4
2023,5
2023,6
2023,7
2023,8
2023,9
2023,10


**Obs.2**.: transações de 2025 excluídas da tabela

### 5.1.3) Clientes distintos por transação
Verificar se há clientes distintos no mesmo InvoiceID. Não pode existir essa condição na base, pois pode indicar dados inconsistentes

In [0]:
%sql
SELECT InvoiceID, COUNT(DISTINCT CustomerID) AS num_customers
FROM fsilver.stransacoes
GROUP BY InvoiceID
HAVING num_customers > 1;

InvoiceID,num_customers


**Obs.3:** Não há transações feitas por mais de um cliente, nesse ponto os dados estão consistentes

### 5.1.4) Chaves ausentes (clientes)
Verificar se os clientes que fizeram transações existem na tabela clientes

In [0]:
%sql
-- contar a quantidade de clientes distintos totais nas transações
SELECT COUNT(DISTINCT CustomerID) AS total_CustomerID_transacoes
FROM fsilver.stransacoes

total_CustomerID_transacoes
1258250


In [0]:
%sql
-- contar CustomerID distintos que estão na interseção entre as 2 tabelas
SELECT COUNT(DISTINCT s.CustomerID) AS total_CustomerID_identificavel
FROM fsilver.stransacoes s
INNER JOIN fsilver.sclientes c ON s.CustomerID = c.CustomerID;

total_CustomerID_identificavel
1258250


**Obs.4:** a quantidade de clientes distintos na tabela de transações e na interseção (join) entre as 2 tabelas é a mesma, portanto, todos os clientes que compraram ou devolveram produtos são identificáveis na tabela dimensão clientes

### 5.1.5) Chaves ausentes (produtos)
Verificar se todos os produtos vendidos existem na tabela produtos. O racional é o mesmo do item 5.1.4

In [0]:
%sql
SELECT COUNT(DISTINCT ProductID) AS total_ProductID_transacoes
FROM fsilver.stransacoes

total_ProductID_transacoes
16266


In [0]:
%sql
SELECT COUNT(DISTINCT s.ProductID) AS total_ProductID_identificavel
FROM fsilver.stransacoes s
INNER JOIN fsilver.sprodutos c ON s.ProductID = c.ProductID

total_ProductID_identificavel
16266


**Obs.5:** todos os produtos comprados ou devolvidos são identificáveis na tabela dimensão produtos

### 5.1.6) Telefone e e-mail por cliente
Identificar se clientes distintos têm o mesmo telefone ou e-mail. Isso não pode acontecer, pois cada pessoa tem um telefone e e-mail

In [0]:
%sql
--verificando se o mesmo e-mail pertence a clientes diferentes
SELECT Email, COUNT(DISTINCT CustomerID) AS num_clientes
FROM fsilver.sclientes
GROUP BY Email
HAVING num_clientes > 1
LIMIT 10

Email,num_clientes
tony.seifert@fake_aol.de,2
danielle.black@fake_gmail.com,2
william.harper@fake_yahoo.com,4
jacqueline.taylor@fake_yahoo.com,3
kevin.thornton@fake_hotmail.com,2
carl.bell@fake_yahoo.com,2
nicholas.clark@fake_gmail.com,6
thomas.miller@fake_gmail.com,19
anthony.armstrong@fake_hotmail.com,2
erin.flores@fake_yahoo.com,2


In [0]:
%sql
--entendendo a quantidade de registros na tabela (CustomerID distintos) clientes na situação acima
SELECT COUNT(*) AS total_registros_inconsistentes_email
FROM fsilver.sclientes
WHERE Email IN (
    SELECT Email FROM fsilver.sclientes GROUP BY Email HAVING COUNT(DISTINCT CustomerID) > 1
)

total_registros_inconsistentes_email
692263


In [0]:
%sql
--entendendo agora a quantidade de e-mails nessa situação
SELECT COUNT(*) AS total_emails_inconsistentes
FROM (
    SELECT Email FROM fsilver.sclientes GROUP BY Email HAVING COUNT(DISTINCT CustomerID) > 1
) AS emails_repetidos

total_emails_inconsistentes
161121


In [0]:
%sql
--vendo alguns dos casos em que o mesmo email pertence a clientes distintos
SELECT *
FROM fsilver.sclientes
WHERE Email IN (
  'tony.seifert@fake_aol.de', 'william.harper@fake_yahoo.com', 'kayla.hall@fake_gmail.com'
)
ORDER BY Name, DateOfBirth ASC

CustomerID,Name,Email,Telephone,City,Country,Gender,DateOfBirth,JobTitle
183375,Kayla Hall,kayla.hall@fake_gmail.com,(275)977-1783,Chicago,United States,F,1975-01-30T00:00:00.000+0000,Event organiser
176647,Kayla Hall,kayla.hall@fake_gmail.com,(830)325-4994x4322,Chicago,United States,F,2003-10-17T00:00:00.000+0000,null
187466,Kayla Hall,kayla.hall@fake_gmail.com,273-218-4855,Chicago,United States,F,2004-01-15T00:00:00.000+0000,null
720395,Tony Seifert,tony.seifert@fake_aol.de,+49(0)9160 693338,Berlin,Deutschland,M,1996-12-06T00:00:00.000+0000,Technical author
896371,Tony Seifert,tony.seifert@fake_aol.de,01671 39429,Leipzig,Deutschland,M,2005-10-15T00:00:00.000+0000,null
216007,William Harper,william.harper@fake_yahoo.com,876.787.0312x6103,Houston,United States,M,1980-09-21T00:00:00.000+0000,Consulting civil engineer
116973,William Harper,william.harper@fake_yahoo.com,+1-685-251-3227,Los Angeles,United States,M,1985-04-12T00:00:00.000+0000,"Administrator, education"
1030486,William Harper,william.harper@fake_yahoo.com,802.381.3206x325,Liverpool,United Kingdom,M,1996-12-05T00:00:00.000+0000,Merchant navy officer
187306,William Harper,william.harper@fake_yahoo.com,6942060958,Chicago,United States,M,2005-02-09T00:00:00.000+0000,null


In [0]:
%sql
--verificar se os casos em que o mesmo e-mail pertence a mais de um cliente são casos de homônimos, verificando se as datas de nascimento são diferentes
SELECT Email, COUNT(DISTINCT DateOfBirth) AS num_datas_nascimento
FROM fsilver.sclientes
WHERE Email IN (
    SELECT Email FROM fsilver.sclientes GROUP BY Email HAVING COUNT(DISTINCT CustomerID) > 1
)
GROUP BY Email
HAVING num_datas_nascimento = 1
LIMIT 10

Email,num_datas_nascimento
linda.arnold@fake_gmail.com,1
vincent.marty@fake_wanadoo.fr,1
康琳@fake_hotmail.com,1
武娜@fake_yahoo.com,1
谭鑫@fake_hotmail.com,1
郎兰英@fake_gmail.com,1
mary.lloyd@fake_gmail.com,1
michelle.fields@fake_hotmail.com,1
ryan.lowe@fake_gmail.com,1
katie.murphy@fake_gmail.com,1


Como há pouquíssimos casos em que a data de nascimento do cliente é a mesma, infere-se que se trata de homônimos, ou seja, não se trata do mesmo cliente, vamos atualizar a coluna Email da tabela para NULL para lidar com essa inconsistência

In [0]:
%sql
--verificando a quantidade de registros com Email = NULL antes de atualizá-los para comparar a quantidade atualizada depois

In [0]:
%sql
SELECT COUNT(CustomerID) FROM fsilver.sclientes WHERE Email IS NULL

count(CustomerID)
0


In [0]:
%sql
UPDATE fsilver.sclientes 
SET Email = NULL 
WHERE Email IN (
    SELECT Email FROM fsilver.sclientes GROUP BY Email HAVING COUNT(DISTINCT CustomerID) > 1
)

num_affected_rows
692263


In [0]:
%sql
--verificando a quantidade de Email NULL depois da atualização
SELECT COUNT(CustomerID) FROM fsilver.sclientes WHERE Email IS NULL

count(CustomerID)
692263


Fazendo a mesma verificação feita para o e-mail, mas agora para telefone

In [0]:
%sql
--verificar se o mesmo telefone pertence a clientes diferentes
SELECT Telephone, COUNT(DISTINCT CustomerID) AS num_clientes
FROM fsilver.sclientes
GROUP BY Telephone
HAVING num_clientes > 1
LIMIT 10

Telephone,num_clientes
+34 820972730,2
15690646361,2
+351962248322,2
08093 84040,2
+34 823 407 769,2
02 48 26 65 08,2
03 57 29 27 44,2
(351) 919 379 652,2
18224980430,2
18913346308,2


In [0]:
%sql
--verificar se os casos em que o mesmo telefone pertence a mais de um cliente são casos de homônimos, através da checagem de datas de nascimento diferentes
SELECT Telephone, COUNT(DISTINCT DateOfBirth) AS num_datas_nascimento
FROM fsilver.sclientes
WHERE Telephone IN (
    SELECT Telephone FROM fsilver.sclientes GROUP BY Telephone HAVING COUNT(DISTINCT CustomerID) > 1
)
GROUP BY Telephone
HAVING num_datas_nascimento = 1

Telephone,num_datas_nascimento


Não há casos em que a data de nascimento dos clientes é a mesma. Admite-se que se trata de homônimo. Será usada a mesma abordagem usada para o tratamento de e-mails iguais, deixando os telefones como NULL

In [0]:
%sql
--verificando a quantidade de Telefones NULL antes da atualização
SELECT COUNT(CustomerID) FROM fsilver.sclientes WHERE Telephone IS NULL

count(CustomerID)
0


In [0]:
%sql
UPDATE fsilver.sclientes 
SET Telephone = NULL 
WHERE Telephone IN (
    SELECT Telephone FROM fsilver.sclientes GROUP BY Telephone HAVING COUNT(DISTINCT CustomerID) > 1
)

num_affected_rows
216


In [0]:
%sql
--verificando a quantidade de Telefones NULL depois da atualização
SELECT COUNT(CustomerID) FROM fsilver.sclientes WHERE Telephone IS NULL

count(CustomerID)
216


## 5.2) Respostas das questões levantadas

In [0]:
%sql DROP DATABASE IF EXISTS fgold CASCADE;

In [0]:
%sql CREATE DATABASE fgold;

In [0]:
# Salvando as tabelas no banco de dados fgold

# Tabela fato: gfato_transacoes
spark.table("fsilver.stransacoes").write.mode("overwrite").saveAsTable("fgold.fato_transacoes")

# Tabelas dimensão copiadas diretamente da cama silver, já foram tratadas e atualizadas com SQL nos casos em que foi preciso
spark.table("fsilver.sclientes").write.mode("overwrite").saveAsTable("fgold.dim_clientes")
spark.table("fsilver.sprodutos").write.mode("overwrite").saveAsTable("fgold.dim_produtos")
spark.table("fsilver.slojas").write.mode("overwrite").saveAsTable("fgold.dim_lojas")
spark.table("fsilver.sfuncionarios").write.mode("overwrite").saveAsTable("fgold.dim_funcionarios")

In [0]:
%sql
SELECT p.Category, COUNT(t.ProductID) AS qtdade_vendida
FROM fgold.fato_transacoes t LEFT JOIN fgold.dim_produtos p
  ON t.ProductID = p.ProductID
WHERE TransactionType = 'Sale'
GROUP BY p.Category
ORDER BY qtdade_vendida DESC

Category,qtdade_vendida
Feminine,2717872
Masculine,2230861
Children,696294


A categoria de produtos mais vendidos foi a Feminina, com 2.717.872 produtos vendidos no período inteiro de análise

In [0]:
%sql
SELECT date_format(`Date`,'yyyy-MM') AS year_month, COUNT(DISTINCT(InvoiceID)) AS vendas
FROM fgold.fato_transacoes 
WHERE TransactionType = 'Sale'
GROUP BY date_format(`Date`,'yyyy-MM')
ORDER BY vendas DESC

year_month,vendas
2024-12,494169
2023-12,450186
2024-09,241966
2024-03,213264
2023-09,209295
2024-10,203170
2023-10,184888
2023-03,177969
2024-05,152786
2024-11,151080


O mês com mais vendas foi dezembro/2024 com 494.169 transações desse tipo

Em abril de 2024, ano mais recente, há 15 clientes com 2 devoluções, que é a maior quantidade de devoluções desse período

# 6) Autoavaliação
## 6.1) Escolha dos dados
Inicialmente, pensou-se em usar dados da empresa em que trabalho para aproveitar o conhecimento do negócio relacionado ao conjunto de dados porém, não estava sendo um processo simples de ter a autorização. Desse modo, foi escolhido um tema de interesse e iniciada a busca no Kaggle, por ser conhecido amplamente pela quantidade de dados disponível para pessoas que querem aprender mais sobre análise e ciência de dados. Nesse contexto, entende-se que, é muito provável que quanto mais conhecimento do negócio se tem, melhores são as perguntas, que consequentemente trazem respostas melhores e análises com mais qualidade para o negócio.

## 6.2) Tratamento da qualidade dos dados
Como a base já era muito bem tratada, poucos tratamentos foram feitos em relação a camada bronze. Nos tratamentos feitos, como por exemplo, e-mail iguais para clientes a princípio distintos, ficou claro que pode existir um erro no processo de cadastro de clientes na empresa fictícia, que deve ser tratado no processo de cadastro. Além disso, o tratamento é essencial para ter segurança ao se trabalhar com os dados.

## 6.3) Respostas das perguntas levantadas como objetivos
Nem todas as perguntas foram respondidas, nota-se que as mais básicas foram respondidas por falta de tempo.

## 6.4) Uso de SQL ou Python
Inúmeras vezes o SQL se mostrou extremamente simples de se chegar a resposta principalmente na etapa de análise da qualidade dos dados, na passagem da camada silver para a gold. Ficou entendido que mesmo sendo uma linguagem "limitada" a consultas, no bom sentido, se mostra mais simples e dá menos trabalho do que usar uma linguagem de programação, contrariando meu pensamento inicial e eliminando um possível pré-conceito de que Python é melhor do que SQL.
